In [6]:
from sklearn.metrics import fbeta_score, make_scorer, ConfusionMatrixDisplay, classification_report, confusion_matrix
from utils.data_loader import load_split, prepare_features_target
import optuna
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from utils.learning_curve import learning_curve_with_resampling
from imblearn.pipeline import Pipeline
from models.manage_models import save_model, load_model

In [7]:
f2_scorer = make_scorer(fbeta_score, beta=2)

In [8]:
train_df = load_split('train', data_dir='dataset/splits')
X_train, y_train = prepare_features_target(train_df, target_col='Fault')

In [9]:
def objective_rf_sampling(trial, X, y):
    strategy = trial.suggest_categorical("bal_strategy", ["undersample", "internal_weight"])
    
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300), # Bajamos a 300 para ganar velocidad
        "max_depth": trial.suggest_int("max_depth", 5, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 2, 10),
        "n_jobs": -1,
        "random_state": 42
    }

    if strategy == "internal_weight":
        param["class_weight"] = "balanced_subsample"
    else:
        param["class_weight"] = None # El balanceo se hace por el sampler externo

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        if strategy == "undersample":
            rus = RandomUnderSampler(random_state=42)
            X_tr, y_tr = rus.fit_resample(X_tr, y_tr)

        model = RandomForestClassifier(**param)
        model.fit(X_tr, y_tr)
        
        preds = model.predict(X_val)
        scores.append(fbeta_score(y_val, preds, beta=2))

    return np.mean(scores)

In [10]:
study = optuna.create_study(direction="maximize", study_name='randomforest_opt')
study.optimize(lambda trial: objective_rf_sampling(trial, X_train, y_train), n_trials=30, show_progress_bar=True)

[I 2025-12-24 16:40:03,609] A new study created in memory with name: randomforest_opt
Best trial: 0. Best value: 0.757232:   3%|▎         | 1/30 [00:37<18:19, 37.93s/it]

[I 2025-12-24 16:40:41,538] Trial 0 finished with value: 0.7572320650293743 and parameters: {'bal_strategy': 'internal_weight', 'n_estimators': 237, 'max_depth': 20, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.7572320650293743.


Best trial: 1. Best value: 0.763975:   7%|▋         | 2/30 [01:20<18:51, 40.42s/it]

[I 2025-12-24 16:41:23,706] Trial 1 finished with value: 0.7639752628941625 and parameters: {'bal_strategy': 'internal_weight', 'n_estimators': 268, 'max_depth': 18, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.7639752628941625.


Best trial: 1. Best value: 0.763975:  10%|█         | 3/30 [01:29<11:47, 26.19s/it]

...

[I 2025-12-24 16:47:30,656] Trial 27 finished with value: 0.7905629735247964 and parameters: {'bal_strategy': 'undersample', 'n_estimators': 181, 'max_depth': 19, 'min_samples_leaf': 5}. Best is trial 7 with value: 0.7919641097435082.


Best trial: 7. Best value: 0.791964:  97%|█████████▋| 29/30 [07:42<00:13, 13.93s/it]

[I 2025-12-24 16:47:46,527] Trial 28 finished with value: 0.7888422335558016 and parameters: {'bal_strategy': 'undersample', 'n_estimators': 184, 'max_depth': 18, 'min_samples_leaf': 4}. Best is trial 7 with value: 0.7919641097435082.


Best trial: 7. Best value: 0.791964: 100%|██████████| 30/30 [08:03<00:00, 16.13s/it]

[I 2025-12-24 16:48:07,562] Trial 29 finished with value: 0.7386651442905436 and parameters: {'bal_strategy': 'internal_weight', 'n_estimators': 170, 'max_depth': 8, 'min_samples_leaf': 3}. Best is trial 7 with value: 0.7919641097435082.
